In [9]:
train_data = 'data/train.txt'

In [10]:
with open(train_data, 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3520678


In [12]:
print(text[:1000])

Once upon a time there was a mommy and her baby. Mommy wanted to keep the baby safe, so she taught the baby to not touch anything yellow. 
Mommy said, "Baby, please don't touch the yellow because it might be dangerous. It's better to be safe and prevent any harm."
But one day, the baby saw a yellow butterfly fluttering around. The butterfly was so pretty, and it looked harmless.
The baby asked, "Mommy, can I touch the butterfly?"
Mommy said, "Yes, you can, baby. That butterfly is harmless and won't hurt you. But remember, don't touch the yellow things that don't look harmless."
The baby nodded and smiled. From then on, the baby knew to be careful when it came to yellow.
<|endoftext|>

Micky was a funny little mouse. He liked to do things differently. One day, he had an idea for a new game.
Micky went over to his friend Daisy's house. "Let's play a game!", he said. Daisy smiled.
Micky asked Daisy to stand on one foot and balance a juice bottle on her neck. They both had a lot of fun. Mi

In [13]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$',-.0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz|¡«»áéñó–—‘’“”…
91


In [14]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[57, 58, 58, 1, 69, 57, 54, 67, 54]
hii there


In [15]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)

In [16]:
data.shape, data.dtype

(torch.Size([3520678]), torch.int64)

In [19]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [20]:
block_size = 8
train_data[:block_size+1]

tensor([38, 63, 52, 54,  1, 70, 65, 64, 63])

In [21]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([38]) the target: 63
when input is tensor([38, 63]) the target: 52
when input is tensor([38, 63, 52]) the target: 54
when input is tensor([38, 63, 52, 54]) the target: 1
when input is tensor([38, 63, 52, 54,  1]) the target: 70
when input is tensor([38, 63, 52, 54,  1, 70]) the target: 65
when input is tensor([38, 63, 52, 54,  1, 70, 65]) the target: 64
when input is tensor([38, 63, 52, 54,  1, 70, 65, 64]) the target: 63


In [22]:
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 64, 63,  1, 57, 54, 67,  1],
        [55, 67, 58, 54, 63, 53,  8,  3],
        [62,  1, 53, 64, 54, 68,  1, 63],
        [ 1, 61, 64, 50, 53, 54, 53,  1]])
targets:
torch.Size([4, 8])
tensor([[64, 63,  1, 57, 54, 67,  1, 72],
        [67, 58, 54, 63, 53,  8,  3,  0],
        [ 1, 53, 64, 54, 68,  1, 63, 64],
        [61, 64, 50, 53, 54, 53,  1, 72]])
----
when input is [1] the target: 64
when input is [1, 64] the target: 63
when input is [1, 64, 63] the target: 1
when input is [1, 64, 63, 1] the target: 57
when input is [1, 64, 63, 1, 57] the target: 54
when input is [1, 64, 63, 1, 57, 54] the target: 67
when input is [1, 64, 63, 1, 57, 54, 67] the target: 1
when input is [1, 64, 63, 1, 57, 54, 67, 1] the target: 72
when input is [55] the target: 67
when input is [55, 67] the target: 58
when input is [55, 67, 58] the target: 54
when input is [55, 67, 58, 54] the target: 63
when input is [55, 67, 58, 54, 63] the target: 53
when input is [55, 67, 5

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 91])
tensor(4.6985, grad_fn=<NllLossBackward0>)

QñXe?Lw¡mP5Ah5JUoEos2TX¡…8kKDá<kv9“lM—Qnzngeol69.…<Nl¡RfñtD;’n”'GR?v'Lom
HpóF3eoóO¡my<ñbzZozn”'T9gid


In [24]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('data/train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.213083 M parameters
step 0: train loss 4.6437, val loss 4.6457
step 100: train loss 3.4347, val loss 3.4461
step 200: train loss 3.0727, val loss 3.1066
step 300: train loss 2.8683, val loss 2.8958
step 400: train loss 2.7291, val loss 2.7637
step 500: train loss 2.6404, val loss 2.6803
step 600: train loss 2.5763, val loss 2.6154
step 700: train loss 2.5183, val loss 2.5629
step 800: train loss 2.4841, val loss 2.5189
step 900: train loss 2.4542, val loss 2.4807
step 1000: train loss 2.4205, val loss 2.4514
step 1100: train loss 2.3872, val loss 2.4253
step 1200: train loss 2.3612, val loss 2.4043
step 1300: train loss 2.3452, val loss 2.3745
step 1400: train loss 2.3160, val loss 2.3458
step 1500: train loss 2.2987, val loss 2.3366
step 1600: train loss 2.2809, val loss 2.3107
step 1700: train loss 2.2600, val loss 2.2866
step 1800: train loss 2.2461, val loss 2.2765
step 1900: train loss 2.2273, val loss 2.2539
step 2000: train loss 2.2104, val loss 2.2402
step 2100: train loss 2.